# Elements of structured data

There are two basic types of structured data:
## Numeric
* Continuous -> Can take infinite values in an interval (measurement/inference);
* Discrete -> Only ordered integer values (count).
## Categorical
* Nominal -> Classifies without order (classes);
* Ordinal -> Sorts in order (e.g. hierarchy);
* Binary -> Can take 2 values - 0/1, true/false.

Pandas data types: https://oreil.ly/UGX-4 <br>
SQL data types: https://oreil.ly/cThTM

## Rectangular data
The most important data structure in data science is the rectangular (tabular), in this case, a two-dimensional matrix with rows (records) and columns (variables) with an index to identify the record. <br>
In an analysis there a feature (inputs to analyze) and outcomes (analysis results/predict). <br>
But, in some cases, the data could be nonrectangular like time series, spatial data structures, graphs...

## Estimates of location

### Mean
The most basic estimate value is the mean, the sum of all values divided by the number of values.
$$
x = \frac{\sum_{i=1}^{n} xi}{n}
$$
* NOTE: The x represents the average of the entire population and xbar of the sample. Just as N represents the number of values in the population and n in the sample. <br>

The weighted mean is the sum of all values multiplied by their weights divided by the sum of weights. Is useful when some value is more variable than other or some group is underrepresented.
$$
xw = \frac{\sum_{i=1}^{n} xi*wi}{\sum_{i=1}^{n} wi}
$$
The trimmed mean is mean without the p-smallest and p-biggest values (sorted). This is estimate eliminates the influence of extreme values.
$$
xt = \frac{\sum_{i=p+1}^{n-p} xi}{n - 2p}
$$
Trimmed weighted mean:
$$
xt = \frac{\sum_{i=p+1}^{n-p} xi * wi}{\sum_{i=p+1}^{n-p} wi}
$$


### Median and robust estimates ...